<a href="https://colab.research.google.com/github/Adith-blusim/bp_training_1hr/blob/main/bp_training_1hr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from scipy.interpolate import PchipInterpolator
try:
  from PyEMD import EMD
except:
  !pip install EMD-signal
  from PyEMD import EMD
from PyEMD import EEMD
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter, hilbert
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, Flatten, Dense, BatchNormalization
import seaborn as sns


fs = 490
lowcut = 0.5
highcut = 6.0

def interpolate(arr):
    pchip_interpolator = PchipInterpolator(np.arange(len(arr)), arr)
    x_interp = np.linspace(0, len(arr) - 1, expected_sps)
    y_interp = pchip_interpolator(x_interp)
    return y_interp.tolist()

def butter_bandpass(lowcut, highcut, fs, order=3):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a
def butter_bandpass_filter(data, lowcut, highcut, fs, order=3):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def imf(arr):
  _out = []
  for i in arr:
    _out.append(EMD().emd(i, (np.arange(0, len(i)) / fs))[0])
  return _out

def inp(arr):
  _out = []
  for i in imf1c1:
    _out.append(np.angle(hilbert(i)))
  return _out

## 1 - Read data

# Mat data
rdf = pd.read_csv("one_hr_12-32_BP.csv") # read the raw mat data file - raw df
rdf = rdf.dropna()
rdf = rdf.reset_index(drop=True) #dropped index needs to be reset

# ECG Ground truth data
gdf = pd.read_csv("roi_data.csv") #ground df
gdf.drop(['ECG'],axis=1,inplace=True) # remove the HR data as it is not needed
gdf[['Systolic', 'Diastolic']] = gdf[['Systolic', 'Diastolic']].replace('', np.nan) #replacing empty values with np.nan
gdf = gdf.dropna()
gdf = gdf.reset_index(drop=True)

# print(gdf, rdf)

In [ ]:
## 2- Preprocess data - changing to HMS format and setting timestamp as index
gdf['Timestamp'] = pd.to_datetime(gdf['Timestamp'], format='%Y-%m-%d %H:%M:%S') # Convert string to datetime format
gdf.set_index('Timestamp', inplace=True) #set 'Time' column as index, inplace=true changes original rdf
gdf = gdf.resample('T').ffill() # BP is measured every 2 mins but I am making per-second data, so filling BP value seconds-wise, T is minutes, 5T is every 5 minutes
gdf.index = gdf.index.strftime('%H:%M') # reduce the datetime format to string - the resample needs to be done when index is in datetime format
gdf['Systolic'] = pd.to_numeric(gdf['Systolic'], errors='coerce') # Convert OCR string data to float
gdf['Diastolic'] = pd.to_numeric(gdf['Diastolic'], errors='coerce')

rdf['Timestamp'] = pd.to_datetime(rdf['Timestamp'], format='%Y-%m-%d %H:%M:%S') # Convert string to datetime format
rdf.set_index('Timestamp', inplace=True) #set 'Time' column as index, inplace=true changes original rdf
rdf.index = rdf.index.strftime('%H:%M') # reduce the datetime format
rdf = rdf.groupby('Timestamp').agg(lambda x: x.tolist()).reset_index() #groups values in each channel with the same timestamp value together
rdf.set_index('Timestamp', inplace=True) #was needed as the index was changed to 0,1,2 - have to understand why

# print(gdf,rdf)

In [ ]:
## 3- Merge
merged = pd.merge(rdf, gdf, left_index=True, right_index=True)

# print(merged)

In [ ]:
# 4 - Interpolate after merge
expected_sps = 0
for i in range(5):
  expected_sps += len(merged['Channel1'][np.random.randint(0,merged.shape[0])])
expected_sps = int(expected_sps/5)
print(expected_sps)
interpolated = merged[['Channel1','Channel2','Channel3','Channel4']].applymap(interpolate)
# print(master)

29400


In [ ]:
# 5 - Filtering, IMF - Intrinsic mode function and INP - Instantaneous phase - Butterworth band-pass filter
filtered = pd.DataFrame()
filtered['buttered_channel1'] = [butter_bandpass_filter(data, lowcut, highcut, fs, order=3) for data in interpolated['Channel1']]
filtered['buttered_channel2'] = [butter_bandpass_filter(data, lowcut, highcut, fs, order=3) for data in interpolated['Channel2']]
filtered['buttered_channel3'] = [butter_bandpass_filter(data, lowcut, highcut, fs, order=3) for data in interpolated['Channel3']]
filtered['buttered_channel4'] = [butter_bandpass_filter(data, lowcut, highcut, fs, order=3) for data in interpolated['Channel4']]

imf1c1 = imf(filtered['buttered_channel1'])
imf1c2 = imf(filtered['buttered_channel2'])
imf1c3 = imf(filtered['buttered_channel3'])
imf1c4 = imf(filtered['buttered_channel4'])


ins_phase_c1 = inp(imf1c1)
ins_phase_c2 = inp(imf1c2)
ins_phase_c3 = inp(imf1c3)
ins_phase_c4 = inp(imf1c4)

In [ ]:
# 6 - Training and test data preparation
merged_ipc = np.stack((ins_phase_c1,ins_phase_c2,ins_phase_c3,ins_phase_c4), axis=1)
merged_ipc = np.swapaxes(merged_ipc, 1, 2)
print(merged_ipc.shape)

X_train = merged_ipc
Y_train = merged[['Systolic', 'Diastolic']].to_numpy().astype('float32')
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42) #random state - splits randomly (multiple splits)

In [ ]:
# 7 - Model architecture

input_shape =  (expected_sps, 4)
model = Sequential()

#first Conv layer
model.add(Conv1D(filters=100, kernel_size=21, strides=1, input_shape=input_shape))
model.add(BatchNormalization())
model.add(tf.keras.layers.ReLU())
model.add(MaxPooling1D(pool_size=2))

#second Conv layer
model.add(Conv1D(filters=200, kernel_size=5))
model.add(BatchNormalization())
model.add(tf.keras.layers.ReLU())
model.add(MaxPooling1D(pool_size=2))

#third Conv layer
model.add(Conv1D(filters=300, kernel_size=5))
model.add(BatchNormalization())
model.add(tf.keras.layers.ReLU())
model.add(MaxPooling1D(pool_size=2))

#GlobalAveragePooling1D layer
model.add(GlobalAveragePooling1D())
#Final dense layer
model.add(Dense(2, activation='linear'))

In [ ]:
# 8 - Model training
tf.keras.backend.clear_session()
model.compile(loss=tf.keras.losses.mean_squared_error, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))
model.summary()
history = model.fit(X_train, Y_train, epochs=50, validation_data=(X_test, Y_test), batch_size=64)

In [ ]:
# 9 - Prediction and testing
predictions = model.predict(X_test)
combined = pd.DataFrame(predictions, columns=['Systolic(predicted)', 'Diastolic(predicted)'])
combined[['Systolic(actual)', 'Diastolic(actual)']] = pd.DataFrame(Y_test, columns=['Systolic(actual)', 'Diastolic(actual)'])
print(combined)

In [ ]:
# 10 - Plot - Loss
plt.figure(figsize=(10, 6))
# Training loss
plt.plot(history.history['loss'], label='Training Loss', marker='o')
# Validation loss
plt.plot(history.history['val_loss'], label='Validation Loss', marker='o')
# Adding labels and title
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Performance')
plt.legend()

# Adding grid for better readability
plt.grid(True)
# Show plot
plt.show()

In [ ]:
# 11 - Plot - Compare predicted vs actual
sample_indices = np.arange(len(combined))
plt.figure(figsize=(70,10))
# Plotting line graph for output 1
plt.plot(sample_indices, combined['Systolic(actual)'], label='Systolic(actual)', linestyle='-', color='blue')
plt.plot(sample_indices, combined['Systolic(predicted)'], label='Systolic(predicted)', linestyle='--', color='cyan')

# Plotting line graph for output 2
plt.plot(sample_indices, combined['Diastolic(actual)'], label='Diastolic(actual)', linestyle='-', color='green')
plt.plot(sample_indices, combined['Diastolic(predicted)'], label='Diastolic(predicted)', linestyle='--', color='lime')

# Set labels and title
plt.xlabel('Sample Index')
plt.ylabel('Output Values')
plt.title('Comparison of Predicted and True Outputs')

# Show legend
plt.legend()

# Show the plot
plt.show()

End of the working code


In [ ]:
predictions = model.predict(X_test)
yf = pd.DataFrame(predictions, columns=['Systolic(predicted)', 'Diastolic(predicted)'])
yf[['Systolic(actual)', 'Diastolic(actual)']] = pd.DataFrame(Y_test, columns=['Systolic(actual)', 'Diastolic(actual)'])
yf

In [ ]:

# Plotting training and validation loss


In [ ]:
# 5 - Filtering - Butterworth band-pass filter
filtered = pd.DataFrame()
filtered['buttered_channel1'] = [butter_bandpass_filter(data, lowcut, highcut, fs, order=3) for data in interpolated['Channel1']]
filtered['buttered_channel2'] = [butter_bandpass_filter(data, lowcut, highcut, fs, order=3) for data in interpolated['Channel2']]
filtered['buttered_channel3'] = [butter_bandpass_filter(data, lowcut, highcut, fs, order=3) for data in interpolated['Channel3']]
filtered['buttered_channel4'] = [butter_bandpass_filter(data, lowcut, highcut, fs, order=3) for data in interpolated['Channel4']]

emd = EMD()
imf1c1 = [] # intrensic mode function 1 for channel 1
imf1c2 = [] # intrensic mode function 1 for channel 2
imf1c3 = []
imf1c4 = []

for i in filtered['buttered_channel1']:
    time = np.arange(0, len(i)) / fs/2
    IMFs = emd.emd(i, time)
    i1 = IMFs[0]
    imf1c1.append(i1)

for i in filtered['buttered_channel2']:
    time = np.arange(0, len(i)) / fs
    IMFs = emd.emd(i, time)
    i1 = IMFs[0]
    imf1c2.append(i1)

for i in filtered['buttered_channel3']:
    time = np.arange(0, len(i)) / fs
    IMFs = emd.emd(i, time)
    i1 = IMFs[0]
    imf1c3.append(i1)

for i in filtered['buttered_channel4']:
    time = np.arange(0, len(i)) / fs
    IMFs = emd.emd(i, time)
    i1 = IMFs[0]
    imf1c4.append(i1)

# intrensic phase for channel 1,2,3,4
ins_phase_c1 = []
ins_phase_c2 = []
ins_phase_c3 = []
ins_phase_c4 = []

for i in imf1c1:
    ins_phase_c1.append(np.angle(hilbert(i)))

for i in imf1c2:
    ins_phase_c2.append(np.angle(hilbert(i)))

for i in imf1c3:
    ins_phase_c3.append(np.angle(hilbert(i)))

for i in imf1c4:
    ins_phase_c4.append(np.angle(hilbert(i)))